In [73]:
from arcgis.gis import GIS
from arcgis.geometry import Envelope, Point
from arcgis.features import Feature, FeatureSet
from datetime import datetime
from data_engineering.utils import fetch_charging_stations, fetch_hotcold_features, fetch_traffic_accidents, get_hotcold_layer, get_live_traffic_item
from urban_traffic.utils import create_map, create_traffic_map, fetch_traffic_data, filter_commute_cars, generate_car_renderer, prepare_traffic, read_traffic_features
import os
import pandas as pd

### Additional dependencies
We load the necessary variables using `dotenv`.  
Please, ensure that you setup before: `pip install python-dotenv`  
Furthermore, put a `.env` file into the root of this repository.

In [145]:
from dotenv import load_dotenv

def load_vars():
    load_dotenv(dotenv_path="../../../../.env", override=True)
    return os.getenv("ARCGIS_API_KEY"), os.getenv("TRAFFIC_DATA_FILE"), os.getenv("TRAFFIC_FEATURES"), os.getenv("NETWORK_DATASET"), os.getenv("ELECTRIC_CAR_FEATURES")

In [146]:
api_key, traffic_data_filepath, traffic_features_filepath, network_dataset_filepath, electric_car_features_filepath = load_vars()

### Connect to your GIS portal

In [6]:
gis = GIS(api_key=api_key)

### Use a map for the area of interest

In [7]:
traffic_map_view = create_traffic_map(gis)
traffic_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

In [8]:
map_view = create_map(gis)
map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

### Spatial filter using the area of interest

In [9]:
distance_in_meters = 150
lon, lat = map_view.center[:2]
traffic_features = read_traffic_features(traffic_features_filepath, lon=lon, lat=lat, meters=distance_in_meters)
traffic_features[["trip", "person", "vehicle_type"]]

,trip,person,vehicle_type
0,12069,7524,Bike
1,12069,7524,Bike
2,12069,7524,Bike
3,12069,7524,Bike
4,12069,7524,Bike
...,...,...,...
9026,6263,3892,None
9027,6263,3892,None
9028,6263,3892,None
9029,6263,3892,None


## Ingestion & Data Engineering

In [10]:
traffic_features.dtypes

OBJECTID           int64
SHAPE           geometry
trip               int64
person             int64
vehicle_type      object
trip_time         object
dtype: object

In [13]:
prepared_traffic_features = prepare_traffic(traffic_features.copy())
prepared_traffic_features

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
0,229354,"{""x"": 8.650534751000066, ""y"": 50.1136668510000...",12069,7524,3,50,58,1,0,0
1,229379,"{""x"": 8.650534005000054, ""y"": 50.1136312120000...",12069,7524,3,50,59,1,0,0
2,229404,"{""x"": 8.650538966000056, ""y"": 50.1135955960000...",12069,7524,3,51,0,1,0,0
3,229429,"{""x"": 8.650543927000058, ""y"": 50.1135599790000...",12069,7524,3,51,1,1,0,0
4,229454,"{""x"": 8.650556937000033, ""y"": 50.1135254140000...",12069,7524,3,51,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...
9026,17190362,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,10,0,0,1
9027,17190390,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,11,0,0,1
9028,17190418,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,12,0,0,1
9029,17190446,"{""x"": 8.652244763000056, ""y"": 50.1132610030000...",6263,3892,23,8,13,0,0,1


In [14]:
prepared_traffic_features.dtypes

OBJECTID         int64
SHAPE         geometry
trip             int64
person           int64
hour             int32
minute           int32
second           int32
bike             int32
car              int32
pedestrian       int32
dtype: object

### Visualize commute traffic

In [15]:
commute_cars = filter_commute_cars(prepared_traffic_features.copy())
commute_cars

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
1040,2476833,"{""x"": 8.650609811000038, ""y"": 50.1136165960000...",4772,2962,8,5,37,0,1,0
1041,2476877,"{""x"": 8.650567598000066, ""y"": 50.1136170170000...",11422,7117,8,5,37,0,1,0
1042,2477413,"{""x"": 8.650652800000046, ""y"": 50.1134959590000...",4772,2962,8,5,38,0,1,0
1043,2477481,"{""x"": 8.650606480000022, ""y"": 50.1134955600000...",11422,7117,8,5,38,0,1,0
1044,2477874,"{""x"": 8.650717396000061, ""y"": 50.1133933380000...",11422,7117,8,5,39,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4053,7116251,"{""x"": 8.65206020900007, ""y"": 50.11311437500006...",10887,6787,9,59,50,0,1,0
4054,7116948,"{""x"": 8.652221206000036, ""y"": 50.1130443840000...",10887,6787,9,59,51,0,1,0
4055,7117645,"{""x"": 8.652382205000038, ""y"": 50.1129743940000...",10887,6787,9,59,52,0,1,0
4056,7118341,"{""x"": 8.65254320400004, ""y"": 50.11290440200003...",10887,6787,9,59,53,0,1,0


In [16]:
commute_map_view = create_map(gis)
commute_cars.spatial.plot(commute_map_view, renderer=generate_car_renderer())
commute_map_view.zoom_to_layer(commute_cars)
commute_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962783.3385332164, 'ymin': 6465530.3265399…

## Advanced Spatial Analysis

In [17]:
hotcold_spots, drawing_info = fetch_hotcold_features(gis, commute_cars)
commute_map_view.content.add(hotcold_spots, drawing_info=drawing_info, options={"opacity": 0.3})

In [38]:
charging_stations = fetch_charging_stations(gis, extent=Envelope(commute_map_view.extent))
charging_stations

,OBJECTID,Ladeeinrichtungs_ID,Betreiber,Anzeigename__Karte_,Status,Art_der_Ladeeinrichtung,Anzahl_Ladepunkte,Nennleistung_Ladeeinrichtung__k,Inbetriebnahmedatum,Straße,...,Public_Key4,Steckertypen5,Nennleistung_Stecker5,EVSE_ID5,Public_Key5,Steckertypen6,Nennleistung_Stecker6,EVSE_ID6,Public_Key6,SHAPE
0,42695,1041597.0,APCOA Deutschland GmbH,<NA>,In Betrieb,Normalladeeinrichtung,2.0,11.0,2016-12-01,Friedrich-Ebert-Anlage,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 963220.8372000009, ""y"": 6465760.8719, ""s..."
1,42696,1106985.0,E.ON Drive GmbH,E.ON Drive GmbH,In Betrieb,Normalladeeinrichtung,1.0,11.0,2020-12-07,Friedrich-Ebert-Anlage,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 963037.7166000009, ""y"": 6465896.09849999..."
2,42697,1106986.0,E.ON Drive GmbH,E.ON Drive GmbH,In Betrieb,Normalladeeinrichtung,1.0,11.0,2020-12-07,Friedrich-Ebert-Anlage,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 963037.7166000009, ""y"": 6465896.09849999..."
3,42698,1107107.0,E.ON Drive GmbH,E.ON Drive GmbH,In Betrieb,Normalladeeinrichtung,1.0,11.0,2021-05-11,Friedrich-Ebert-Anlage,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 963037.7166000009, ""y"": 6465896.09849999..."
4,42699,1107109.0,E.ON Drive GmbH,E.ON Drive GmbH,In Betrieb,Normalladeeinrichtung,1.0,11.0,2020-12-08,Friedrich-Ebert-Anlage,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 963037.7166000009, ""y"": 6465896.09849999..."
5,43017,1112038.0,On Charge GmbH,<NA>,In Betrieb,Normalladeeinrichtung,2.0,27.0,2024-06-06,Bettinaplatz,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 963689.7148999982, ""y"": 6465998.69160000..."
6,43019,1043206.0,Allego GmbH,Allego,In Betrieb,Schnellladeeinrichtung,2.0,150.0,2021-10-13,Brüsseler Straße,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 963181.7639999986, ""y"": 6465486.25949999..."


In [156]:
import arcpy
from arcpy.da import SearchCursor
from arcpy.management import CopyFeatures
from arcpy.na import AddLocations, GetNAClassNames, Solve

In [157]:
arcpy.env.overwriteOutput = True

In [158]:
analysis_layer_name = "ClosestChargingStation"
layer_result = arcpy.na.MakeClosestFacilityAnalysisLayer(network_dataset_filepath, analysis_layer_name)
layer_result

<Result 'GPLYR_{3D16EC1F-C699-4DBB-BC00-80C94FC42BB1}'>

In [159]:
analysis_layer = layer_result.getOutput(0)
analysis_class_names = GetNAClassNames(analysis_layer)
analysis_class_names

{'Facilities': 'Facilities',
 'Incidents': 'Incidents',
 'Barriers': 'Point Barriers',
 'CFRoutes': 'Routes',
 'PolylineBarriers': 'Line Barriers',
 'PolygonBarriers': 'Polygon Barriers'}

In [160]:
input_facilities = "memory/facilities"
charging_stations.spatial.to_featureclass(input_facilities)

'memory\\facilities'

In [161]:
AddLocations(analysis_layer, analysis_class_names["Facilities"], input_facilities)

<Result 'GPLYR_{7073C0D9-DA3C-4A82-9EFC-C5D890A6C3AB}'>

In [162]:
lon, lat = commute_map_view.center[:2]
input_location = Point({
    "x": lon,
    "y": lat,
    "spatialReference": {"wkid": 4326}
})

input_feature = Feature(input_location, {
    "trip_time": datetime.now().isoformat()
})

input_featureset = FeatureSet([input_feature])
input_featureset.sdf

,trip_time,OBJECTID,SHAPE
0,2025-11-12T19:01:53.946161,1,"{""x"": 8.652844211994614, ""y"": 50.1125321748243..."


In [163]:
latest_positions = pd.DataFrame.spatial.from_featureclass(electric_car_features_filepath)
latest_positions

,OBJECTID,SHAPE
0,1,"{""x"": 944663.6695000008, ""y"": 6460319.02390000..."
1,2,"{""x"": 971470.192400001, ""y"": 6457397.931100003..."
2,3,"{""x"": 977864.9254000001, ""y"": 6470463.21670000..."
3,4,"{""x"": 972219.3685999997, ""y"": 6480431.19870000..."
4,5,"{""x"": 943752.0808000006, ""y"": 6475590.21429999..."


In [164]:
input_incidents = "memory/incidents"
#input_featureset.sdf.spatial.to_featureclass(input_incidents)
latest_positions.spatial.to_featureclass(input_incidents)
AddLocations(analysis_layer, analysis_class_names["Incidents"], input_incidents)

<Result 'GPLYR_{7073C0D9-DA3C-4A82-9EFC-C5D890A6C3AB}'>

In [165]:
solve_result = Solve(analysis_layer)
solve_result

<Result 'GPLYR_{7073C0D9-DA3C-4A82-9EFC-C5D890A6C3AB}'>

In [166]:
result_layer = solve_result.getOutput(0)
result_layer

In [167]:
with SearchCursor(result_layer, "*") as cursor:
    for row in cursor:
        print(row)

(1, (963220.8372000009, 6465760.8719), 'Location 1', 1, 40565249, 0.5936181710860426, 2, 0, 0, 963240.2495009071, 6465790.65992697, 0.0, 35.555083704492844, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None, None, None, None, None, None, None, None)
(2, (963037.7166000009, 6465896.098499998), 'Location 2', 1, 40565191, 0.4423928998685181, 1, 0, 0, 963036.7485363628, 6465894.856919229, 0.0, 1.574379247311541, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None, None, None, None, None, None, None, None)
(3, (963037.7166000009, 6465896.098499998), 'Location 3', 1, 40565191, 0.4423928998685181, 1, 0, 0, 963036.7485363628, 6465894.856919229, 0.0, 1.574379247311541, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None, None, None, None, None, None, None, None)
(4, (963037.7166000009, 6465896.098499998), 'Location 4', 1, 40565191, 0.4423928998685181, 1, 0, 0, 963036.7485363628, 6465894.856919229, 0.0, 1.574379247311541, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None, None, None, None, None, None, None, None)
(5, (

In [168]:
output_routes = "memory/routes"
CopyFeatures(analysis_class_names["CFRoutes"], output_routes)

<Result 'memory\\routes'>

In [169]:
charging_routes = pd.DataFrame.spatial.from_featureclass(output_routes)
charging_routes

,OBJECTID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,StartTime,EndTime,StartTimeUTC,EndTimeUTC,Total_Minutes,Total_TravelTime,Total_Miles,Total_Kilometers,Total_TimeAt1KPH,Total_WalkTime,Total_TruckMinutes,Total_TruckTravelTime,SHAPE
0,1,2,1,Location 1 - Location 2,1,2,1,NaT,NaT,NaT,NaT,<NA>,13.973726,<NA>,19.908323,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[944661.0929999985, ..."
1,2,6,1,Location 2 - Location 6,2,2,2,NaT,NaT,NaT,NaT,<NA>,16.485861,<NA>,10.185686,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[971478.6953999996, ..."
2,3,6,1,Location 3 - Location 6,1,2,3,NaT,NaT,NaT,NaT,<NA>,22.413488,<NA>,11.026141,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[977871.9464000016, ..."
3,4,2,1,Location 4 - Location 2,2,2,4,NaT,NaT,NaT,NaT,<NA>,15.408266,<NA>,13.789785,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[972217.7173000015, ..."
4,5,2,1,Location 5 - Location 2,1,2,5,NaT,NaT,NaT,NaT,<NA>,16.170483,<NA>,20.563134,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[943749.5025000013, ..."


In [174]:
from arcgis.map.renderers import SimpleRenderer
from arcgis.map.symbols import SimpleMarkerSymbolEsriSMS, SimpleMarkerSymbolStyle, SimpleLineSymbolEsriSLS, SimpleLineSymbolStyle


def generate_routes_renderer():
    return SimpleRenderer(
        symbol=SimpleLineSymbolEsriSLS(
            color=[255, 155, 155, 155],
            width=7.5,
            style=SimpleLineSymbolStyle.esri_sls_solid.value,
        )
    )

In [175]:
charging_map = create_map(gis)
latest_positions.spatial.plot(charging_map, renderer=generate_car_renderer())
charging_stations.spatial.plot(charging_map, renderer=generate_car_renderer())
charging_routes.spatial.plot(charging_map, renderer=generate_routes_renderer())
charging_map

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…